In [1]:
import glob

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.markers import CARETLEFTBASE, CARETRIGHTBASE
from csv_dtypes import column_dtypes

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import svm, linear_model

In [45]:
all_files = glob.glob("../roll_data/*-weatherproc.csv")

df = pd.concat((pd.read_csv(f, dtype=column_dtypes) for f in all_files), ignore_index=True)

seasons = [11, 12, 13, 14, 15]
df = df[df['season'].isin(seasons)]
len(df)

272727

In [46]:
dfc = df.copy()
for exclude_mod in []:
    dfc = dfc[~dfc["batter_mods"].astype(str).str.contains(exclude_mod)]
    dfc = dfc[~dfc["pitcher_mods"].astype(str).str.contains(exclude_mod)]
    dfc = dfc[~dfc["pitching_team_mods"].astype(str).str.contains(exclude_mod)]
    dfc = dfc[~dfc["batting_team_mods"].astype(str).str.contains(exclude_mod)]
    dfc = dfc[~dfc["fielder_mods"].astype(str).str.contains(exclude_mod)]

#Shows Coffee Weather Proc
#for exclude_event in ["Refill", "NoRefill", "Swept", "NoSweep", "Shuffle", "NoShuffle", "Drain", "Siphon", "Swap", "NoSwap", "Allergy", "NoAllergy"]:
    #dfc = dfc[~dfc["event_type"].astype(str).str.contains(exclude_event)]

#Shows Coffee 2 Weather Proc
#for exclude_event in ["Bean", "NoBean", "Swept", "NoSweep", "Shuffle", "NoShuffle", "Drain", "Siphon", "Swap", "NoSwap", "Allergy", "NoAllergy"]:
    #dfc = dfc[~dfc["event_type"].astype(str).str.contains(exclude_event)]

#Shows Flooding Weather Proc
#for exclude_event in ["Refill", "NoRefill", "Bean", "NoBean", "Shuffle", "NoShuffle", "Drain", "Siphon", "Swap", "NoSwap", "Allergy", "NoAllergy"]:
    #dfc = dfc[~dfc["event_type"].astype(str).str.contains(exclude_event)]

#Shows Reverb Weather Proc
#for exclude_event in ["Refill", "NoRefill", "Swept", "NoSweep", "Bean", "NoBean", "Drain", "Siphon", "Swap", "NoSwap", "Allergy", "NoAllergy"]:
    #dfc = dfc[~dfc["event_type"].astype(str).str.contains(exclude_event)]

#Shows Blooddrain Weather Proc - NOTE: FAILURES ARE SIPHON EVENTS ONLY
for exclude_event in ["Refill", "NoRefill", "Swept", "NoSweep", "Bean", "NoBean", "Swap", "NoSwap", "Shuffle", "NoShuffle", "Allergy", "NoAllergy"]:
    dfc = dfc[~dfc["event_type"].astype(str).str.contains(exclude_event)]

#Shows Feedback Weather Proc
#for exclude_event in ["Refill", "NoRefill", "Swept", "NoSweep", "Shuffle", "NoShuffle", "Drain", "Siphon", "Bean", "NoBean", "Allergy", "NoAllergy"]:
    #dfc = dfc[~dfc["event_type"].astype(str).str.contains(exclude_event)]

#Shows Peanut Weather Proc
#for exclude_event in ["Refill", "NoRefill", "Swept", "NoSweep", "Swap", "NoSwap", "Shuffle", "NoShuffle", "Drain", "Siphon", "Bean", "NoBean"]:
    #dfc = dfc[~dfc["event_type"].astype(str).str.contains(exclude_event)]


In [47]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import svm, linear_model


X = dfc[["roll", "ballpark_fortification"]]
y = dfc['passed']

sc = StandardScaler()
X2 = sc.fit_transform(X)

trainedsvm = svm.LinearSVC(dual=False, C=500000, max_iter=5000000, tol=1e-5).fit(X2, y)
predictionsvm = trainedsvm.predict(X2)
confusion_matrix(y,predictionsvm)

array([[40,  0],
       [ 0,  6]], dtype=int64)

In [48]:
coef = np.true_divide(trainedsvm.coef_, sc.scale_)
coef_scaled = coef / coef[0, 0]
coef_list = coef_scaled.tolist()[0]
sorted(zip(coef_list, X.columns), key=lambda x: abs(x[0]), reverse=True)

[(1.0, 'roll'), (0.0020935610124519334, 'ballpark_fortification')]

In [49]:
intercept = trainedsvm.intercept_ - np.dot(coef, sc.mean_)
intercept_scaled = intercept / coef[0, 0]
intercept_scaled

array([-0.00121929])

In [50]:
print(len(dfc[~dfc["passed"]]))
print(len(dfc[dfc["passed"]]))

40
6


In [51]:

dfc["threshold"] = 0.026

In [52]:
x_val = "threshold"
fig, ax = plt.subplots(1, figsize=(10, 6))
ax.scatter(dfc[dfc['passed']][x_val], dfc[dfc['passed']]["roll"], color="blue",
           label="Activate", marker=CARETLEFTBASE)
ax.scatter(dfc[~dfc['passed']][x_val], dfc[~dfc['passed']]["roll"], color="red",
           label="NoActivate", marker=CARETRIGHTBASE)
ax.plot([0, 1], [0, 1])
ax.set_xlabel(x_val)
ax.set_ylabel('roll')
ax.set_title("Weather Procs")
ax.set_xlim([0.024, 0.028])
ax.set_ylim([0.02, 0.03])
ax.legend()

In [53]:
dfc["offset"] = dfc["roll"] - dfc["threshold"]
outliers = dfc[(dfc["passed"] & (dfc["offset"] > 0)) | (~dfc["passed"] & (dfc["offset"] < 0))]

fig, ax = plt.subplots(1)
x_val = "ballpark_fortification"
y_val = "offset"
ax.scatter(outliers[~outliers['passed']][x_val], outliers[~outliers['passed']][y_val], color="red", label="Proc")
ax.scatter(outliers[outliers['passed']][x_val], outliers[outliers['passed']][y_val], color="blue", label="No Proc")
ax.set_xlabel(x_val)
ax.set_ylabel(y_val)
ax.legend()

table = outliers
table[["offset", "weather", "event_type", "season",
       "roll", "threshold",
       "batter_name", "pitcher_name", "batter_mods", "pitcher_mods",
       "batting_team_mods", "pitching_team_mods", "day",  "game_id", "play_count"]].sort_values('offset')


offset             weather event_type  season      roll  threshold  \
109703 -0.025943  Weather.BLOODDRAIN     Siphon      13  0.000057      0.026   
6447   -0.025791  Weather.BLOODDRAIN     Siphon      11  0.000209      0.026   
35689  -0.025670  Weather.BLOODDRAIN     Siphon      12  0.000330      0.026   
59700  -0.025662  Weather.BLOODDRAIN     Siphon      12  0.000338      0.026   
1491   -0.025529  Weather.BLOODDRAIN     Siphon      11  0.000471      0.026   
54637  -0.025501  Weather.BLOODDRAIN     Siphon      12  0.000499      0.026   
54947  -0.025459  Weather.BLOODDRAIN     Siphon      12  0.000541      0.026   
67539  -0.025420  Weather.BLOODDRAIN     Siphon      12  0.000580      0.026   
10777  -0.025381  Weather.BLOODDRAIN     Siphon      11  0.000619      0.026   
10685  -0.025324  Weather.BLOODDRAIN     Siphon      11  0.000676      0.026   
1946   -0.025301  Weather.BLOODDRAIN     Siphon      11  0.000699      0.026   
11432  -0.025170  Weather.BLOODDRAIN     Siphon      11  0.000830      0.026   
1366   -0.025116  Weather.BLOODDRAIN     Siphon      11  0.000884      0.026   
202425 -0.025010  Weather.BLOODDRAIN     Siphon      14  0.000990      0.026   
19432  -0.024994  Weather.BLOODDRAIN     Siphon      11  0.001006      0.026   
266529 -0.024993  Weather.BLOODDRAIN     Siphon      15  0.001007      0.026   
243683 -0.024992  Weather.BLOODDRAIN     Siphon      15  0.001008      0.026   
266749 -0.024908  Weather.BLOODDRAIN     Siphon      15  0.001092      0.026   
232316 -0.024886  Weather.BLOODDRAIN     Siphon      14  0.001114      0.026   
10684  -0.024791  Weather.BLOODDRAIN     Siphon      11  0.001209      0.026   
158379 -0.024742  Weather.BLOODDRAIN     Siphon      13  0.001258      0.026   
12010  -0.024649  Weather.BLOODDRAIN     Siphon      11  0.001351      0.026   
62827  -0.024558  Weather.BLOODDRAIN     Siphon      12  0.001442      0.026   
11354  -0.024557  Weather.BLOODDRAIN     Siphon      11  0.001443      0.026   
4846   -0.024488  Weather.BLOODDRAIN     Siphon      11  0.001512      0.026   
199619 -0.024299  Weather.BLOODDRAIN     Siphon      14  0.001701      0.026   
50925  -0.024167  Weather.BLOODDRAIN     Siphon      12  0.001833      0.026   
99983  -0.024144  Weather.BLOODDRAIN     Siphon      13  0.001856      0.026   
40570  -0.024137  Weather.BLOODDRAIN     Siphon      12  0.001863      0.026   
59866  -0.024124  Weather.BLOODDRAIN     Siphon      12  0.001876      0.026   
69884  -0.024004  Weather.BLOODDRAIN     Siphon      12  0.001996      0.026   
13669  -0.023985  Weather.BLOODDRAIN     Siphon      11  0.002015      0.026   
67654  -0.023825  Weather.BLOODDRAIN     Siphon      12  0.002175      0.026   
1417   -0.023796  Weather.BLOODDRAIN     Siphon      11  0.002204      0.026   
13374  -0.023765  Weather.BLOODDRAIN     Siphon      11  0.002235      0.026   
4903   -0.023724  Weather.BLOODDRAIN     Siphon      11  0.002276      0.026   
39284  -0.023633  Weather.BLOODDRAIN     Siphon      12  0.002367      0.026   
61025  -0.023586  Weather.BLOODDRAIN     Siphon      12  0.002414      0.026   
2265   -0.023558  Weather.BLOODDRAIN     Siphon      11  0.002442      0.026   
6914   -0.023507  Weather.BLOODDRAIN     Siphon      11  0.002493      0.026   

                 batter_name        pitcher_name  \
109703       Summers Preston          Liam Snail   
6447             Sam Scandal    Chambers Simmons   
35689      Ayanna Dumpington       Marco Escobar   
59700        Igneus Delacruz       Yummy Elliott   
1491          Mcdowell Mason     Dunlap Figueroa   
54637         Gloria Bugsnax     Rivers Clembons   
54947           Avila Guzman     Zeboriah Wilson   
67539      Elijah Valenzuela          Sosa Hayes   
10777        Lang Richardson      Yosh Carpenter   
10685           Lenny Spruce       Snyder Briggs   
1946             Ronan Combs         Combs Estes   
11432          Wyatt Quitter     Rivers Clembons   
1366         Valentine Games     Dunlap Figueroa   
2

In [51]:
print(dfc[dfc["passed"]]["roll"].max())
print(dfc[~dfc["passed"]]["roll"].min())

0.02532661704696237
0.026462907893175517
